In [1]:
import sys
sys.path.append("/home/kelab/m6AAIpy2")

In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
import numpy as np
import pandas as pd
from Bio.Seq import Seq
import keras.backend as kb

Using TensorFlow backend.


In [3]:
def one_hot_encode(seq):

    map = np.asarray([[0, 0, 0, 0],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
    seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')

    return map[np.fromstring(seq, np.int8) % 5]

In [4]:
def categorical_crossentropy_2d(y_true, y_pred):
    # Standard categorical cross entropy for sequence outputs

    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10))

In [5]:
context = 10000

In [6]:
paths = ('home/kelab/iM6A/mouseRAC10000_c{}.h5'.format(x) for x in range(1, 6))

In [7]:
models = [load_model(y, custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d}) for y in paths]

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
models

### Read data

In [1]:
import numpy as np
import pandas as pd

In [2]:
Fasta = pd.read_csv("mm10_Fasta_Non_Donor_Acceptor.csv")

In [3]:
Fasta

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,...,LastExonLength,LastIntronLength,SecondToLastLength,Pre,PreSequence,SecondToLastExonSequence,LastIntronSequence,LastExonSequence,Truncation,Seq
0,Wscd2,chr5,+,113490446,113589725,113550934,113588145,9,"113490446,113550422,113558348,113560900,113570...","113490940,113551334,113558463,113561085,113570...",...,1933,10329,201,86816,CACGCTGAACACCCATCAACAAAACTGTCCTTTGTTGGATTTCTTT...,GGTTCAAAGGTGAGCGGGACCACTGGCGAAGCGGGCGGACCATCTG...,GTGACAGGAGGGAGGGTCGTGGACAGTGTGAGGTGCGGGTAATCAG...,AGTGGCCCGAGTTCGTGAGGAACTATGCCCCGTGGTGGGCCACGCA...,GTGACAGGAGGGAGGGTCGTGGACAGTGTGAGGTGCGGGTAATCAG...,GGTTCAAAGCAGAGCGGGACCACTGGCGAAGCGGGCGGACCATCTG...
1,Kl,chr5,+,150952606,150993817,150952716,150991854,5,"150952606,150980609,150982502,150988392,150991...","150953541,150981120,150982771,150989494,150993...",...,2301,2022,1102,35786,GATAATCATTGCTCGTGGGGCGGCGGGAGCGGGGGTGGGCACCGCG...,GTGGACACTACTCTCTCTCAGTTTACTGACCCGAATGTCTATCTGT...,GTGAGACCCCGCCCCCTCAATCCCGGAAATACATCAGAGCGTGTAT...,CCTACGTGTTGGACGACATCAACCTTTGTGGCTACTTTGCGTATTC...,GTGAGACCCCGCCCCCTCAATCCCGGAAATACATCAGAGCGTGTAT...,GTGGACACTACTCTCTCTCAGTTTACTGACCCGAATGTCTATCTGT...
2,Tmx3,chr18,+,90510153,90543267,90510322,90540196,16,"90510153,90511278,90514204,90517393,90519104,9...","90510377,90511333,90514244,90517517,90519150,9...",...,3329,1371,69,28345,CTTCGCTGGGCTACCGGGATCCTGAAGGAGGCCGGCCTAccccgcc...,GCCCAAGGAGGTGATAGCATTTTTCAGAGATTGAAAAGAATAGTAT...,GTAAGTTCTGACAGTATCTTCAATAGCTGAAAAATTTGGGGTTTTC...,TCTATATTCAAGAGCTCTCCACTTATGGGCTGCTTTCTTTTCGGGC...,GTAAGTTCTGACAGTATCTTCAATAGCTGAAAAATTTGGGGTTTTC...,GCCCAAGGAGCAGATAGCATTTTTCAGAGATTGAAAAGAATAGTAT...
3,Arhgap21,chr2,-,20847918,20967721,20848711,20967410,25,"20847918,20853037,20854934,20855353,20855580,2...","20850385,20853069,20855040,20855395,20855706,2...",...,2467,2652,32,114652,GGAAGGCTCTCCGAAAGCTGATTTTTCTCTGGCCTCCGAGTAACAC...,ATTCAGCTACTAGTGACTCCGCAAAATCTAAG,GTAAGTTGTTCTATACAGCTGGAAAACACTTCGTGGTAGCTTTGGG...,GGCTCCTGGGGATCCGGGAAGGATCAGTATAGCAGGGAACTGCTTG...,GTAAGTTGTTCTATACAGCTGGAAAACACTTCGTGGTAGCTTTGGG...,ATTCAGCTACTAGTGACTCCGCAAAATCTAAGCAAAGTTGTTCTAT...
4,Ccnj,chr19,+,40831278,40848572,40832019,40846098,6,"40831278,40831978,40836916,40844659,40845044,4...","40831442,40832088,40837127,40844959,40845204,4...",...,2874,494,160,13766,GTACGGCGAGCCGCCTGCCGGTCCTTTAACAATGGGCGGCGCGAGC...,ATTATGCCTTTCTAAATTATGCACCTTCTTTAGTAGCTGCTGCGTG...,GTAAGCCTCTTTGCTCTTGCATTTATAATTTCTAGTTAGCATTTTC...,TGCTCACGATAACGACGTGAAAGAAGCAAACAAGCAGAGGGGACAG...,GTAAGCCTCTTTGCTCTTGCATTTATAATTTCTAGTTAGCATTTTC...,ATTATGCCTTTCTAAATTATGCACCTTCTTTAGTAGCTGCTGCGTG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2365,Wnt4,chr4,+,137277488,137299726,137277679,137296779,5,"137277488,137289113,137295440,137295659,137296...","137277756,137289349,137295572,137295802,137299...",...,3415,509,143,18171,GGCTCTGGGGCGGCGCTGACAGTCGGGTCCGCCGGGCAGCGGGCGC...,GCTTCCAGTGGTCAGGATGCTCGGACAACATCGCCTATGGCGTAGC...,GTAGAGTGGTACCAACCCCATCGACACTGGGGCAGGGCTGCACGGG...,GCCATCTTGACACACATGCGGGTGGAGTGCAAGTGTCACGGGGTGT...,GTAGAGTGGTACCAACCCCATCGACACTGGGGCAGGGCTGCACGGG...,GCTTCCAGTGGTCAGGATGCTCGGACAACATCGCCTATGGCGTAGC...
2366,Inpp4a,chr1,+,37299890,37410736,37357918,37408259,26,"37299890,37357342,37357814,37358507,37364109,3...","37300118,37357404,37358024,37358552,37364228,3...",...,2610,9206,155,98875,GCGGGGTTGTGCGCTGTGTTCTGAGAGAAGGCGGTGGCAGGATTCG...,GTCTGTCGCCGCCTTAACGGGGTCCGATTCACCAGCTGCAAGAGTG...,GTGAGTGAGGCTGCTGCCAGGCTGCATCTGCCCAGCCACTTGCTCT...,TGAGGGTTGTCGGCGAGAAAACACAATGAAGAATGTTGGAAGTCGC...,GTGAGTGAGGCTGCTGCCAGGCTGCATCTGCCCAGCCACTTGCTCT...,GTCTGTCGCCGCCTTAACGGGGTCCGATTCACCAGCTGCAAGAGTG...
2367,Trps1,chr15,-,50654751,50889691,50660674,50846940,6,"50654751,50664686,50822104,50830828,50846013,5...","50661736,50664809,50822708,50831958,50846942,5...",...,6985,2950,123,224882,TGCAAACAACAGAAGACGGTTCATGGCTCTGGCCGCCGCGCCACCA...,AGGCGAAGAGGATCTGGTGTTTTTTGTGCCAATTGCCTGACCACAA...,GTAAGAACTTATCCTAGCCCTTTAGGGGAA

### Select positive strand

In [4]:
Fasta_Pos = Fasta[Fasta["strand"]=="+"]

In [5]:
Fasta_Pos = Fasta_Pos.reset_index(drop = True)

In [6]:
Fasta_Pos

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,...,LastExonLength,LastIntronLength,SecondToLastLength,Pre,PreSequence,SecondToLastExonSequence,LastIntronSequence,LastExonSequence,Truncation,Seq
0,Wscd2,chr5,+,113490446,113589725,113550934,113588145,9,"113490446,113550422,113558348,113560900,113570...","113490940,113551334,113558463,113561085,113570...",...,1933,10329,201,86816,CACGCTGAACACCCATCAACAAAACTGTCCTTTGTTGGATTTCTTT...,GGTTCAAAGGTGAGCGGGACCACTGGCGAAGCGGGCGGACCATCTG...,GTGACAGGAGGGAGGGTCGTGGACAGTGTGAGGTGCGGGTAATCAG...,AGTGGCCCGAGTTCGTGAGGAACTATGCCCCGTGGTGGGCCACGCA...,GTGACAGGAGGGAGGGTCGTGGACAGTGTGAGGTGCGGGTAATCAG...,GGTTCAAAGCAGAGCGGGACCACTGGCGAAGCGGGCGGACCATCTG...
1,Kl,chr5,+,150952606,150993817,150952716,150991854,5,"150952606,150980609,150982502,150988392,150991...","150953541,150981120,150982771,150989494,150993...",...,2301,2022,1102,35786,GATAATCATTGCTCGTGGGGCGGCGGGAGCGGGGGTGGGCACCGCG...,GTGGACACTACTCTCTCTCAGTTTACTGACCCGAATGTCTATCTGT...,GTGAGACCCCGCCCCCTCAATCCCGGAAATACATCAGAGCGTGTAT...,CCTACGTGTTGGACGACATCAACCTTTGTGGCTACTTTGCGTATTC...,GTGAGACCCCGCCCCCTCAATCCCGGAAATACATCAGAGCGTGTAT...,GTGGACACTACTCTCTCTCAGTTTACTGACCCGAATGTCTATCTGT...
2,Tmx3,chr18,+,90510153,90543267,90510322,90540196,16,"90510153,90511278,90514204,90517393,90519104,9...","90510377,90511333,90514244,90517517,90519150,9...",...,3329,1371,69,28345,CTTCGCTGGGCTACCGGGATCCTGAAGGAGGCCGGCCTAccccgcc...,GCCCAAGGAGGTGATAGCATTTTTCAGAGATTGAAAAGAATAGTAT...,GTAAGTTCTGACAGTATCTTCAATAGCTGAAAAATTTGGGGTTTTC...,TCTATATTCAAGAGCTCTCCACTTATGGGCTGCTTTCTTTTCGGGC...,GTAAGTTCTGACAGTATCTTCAATAGCTGAAAAATTTGGGGTTTTC...,GCCCAAGGAGCAGATAGCATTTTTCAGAGATTGAAAAGAATAGTAT...
3,Ccnj,chr19,+,40831278,40848572,40832019,40846098,6,"40831278,40831978,40836916,40844659,40845044,4...","40831442,40832088,40837127,40844959,40845204,4...",...,2874,494,160,13766,GTACGGCGAGCCGCCTGCCGGTCCTTTAACAATGGGCGGCGCGAGC...,ATTATGCCTTTCTAAATTATGCACCTTCTTTAGTAGCTGCTGCGTG...,GTAAGCCTCTTTGCTCTTGCATTTATAATTTCTAGTTAGCATTTTC...,TGCTCACGATAACGACGTGAAAGAAGCAAACAAGCAGAGGGGACAG...,GTAAGCCTCTTTGCTCTTGCATTTATAATTTCTAGTTAGCATTTTC...,ATTATGCCTTTCTAAATTATGCACCTTCTTTAGTAGCTGCTGCGTG...
4,Ky,chr9,+,102505749,102546239,102506142,102542781,11,"102505749,102509110,102512972,102521288,102523...","102506278,102509173,102513035,102521363,102523...",...,4354,2697,191,33248,TCTTAGGGGATAGATGGTGGCTATGAGGCAGCACTCAGCCCAGCCA...,CTACAATGAATTCTACTTCCTCACCCACCCCGCCCTGTTCATCGAG...,GTAAGGAGAAGCACAAGGCGGAGCCCCAGTGACGGCTGAAGCTCCC...,TCAATGGGAAAGCCACAATCACCATCGAGAGCCGGGCCCCAACACT...,GTAAGGAGAAGCACAAGGCGGAGCCCCAGTGACGGCTGAAGCTCCC...,CTACAATGAATTCTACTTCCTCACCCACCCCGCCCTGTTCATCGAG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,Zfp710,chr7,+,80024813,80092751,80081076,80090504,5,"80024813,80081048,80085950,80086433,80090334,","80024986,80082540,80086142,80086608,80092751,",...,2417,3726,175,61620,ggaggaaggagggaaggggaaggaagagcgcagggaggggagagga...,GTGTGCGGGAAGTCCTTCAACCGCATGTACAACCTGCTGGGTCATA...,GTGGGTGGGCTGTGCGCAGTGGACAGAGCAGGATTGACAGCAGCAT...,ACCCCATGATGGAGCTGGCAGGCCCTGACCCCTCTGAACTTGACAA...,GTGGGTGGGCTGTGCGCAGTGGACAGAGCAGGATTGACAGCAGCAT...,GTGTGCGGGAAGTCCTTCAACCGCATGTACAACCTGCTGGGTCATA...
1185,Ptchd4,chr17,+,42315946,42507741,42316304,42503915,4,"42315946,42316654,42377159,42502281,","42316309,42317240,42377640,42507741,",...,5460,124641,481,61213,CTGCTCCAATGGGCCAGCAGCGCCTTCACCCTTTACGTCTCCCATT...,GATGGGAGGAACAGTTTTATTGGACACCAGCTGGGCGGGGTAGTGG...,GTAACTATCCATCCTTGTGGTAATCGAATTCTTGCCTGTTGGGGGC...,GTCATGGAACTAAAGGAGTATTTGAGCTTCTGTCTGGATGGCGGAG...,GTAACTATCCATCCTTGTGGTAATCGAATTCTTGCCTGTTGGGGGC...,GATGGGAGGAACAGTTTTATTGGACACCAGCTGGGCGGGGTAGTGG...
1186,Wnt4,chr4,+,137277488,137299726,137277679,137296779,5,"137277488,137289113,137295440,137295659,137296...","137277756,137289349,137295572,137295802,137299...",...,3415,509,143,18171,GGCTCTGGGGCGGCGCTGACAGTCGGGTCCGCCGGGCAGCGGGCGC...,GCTTCCAGTGGTCAGGATGCTCGGACAACATCGCCTATGGCGTAGC...,GTAGAGTGGTACCAACCCCATCGACACTGGGGCAGGGCTGCACGGG..

In [7]:
PreSequence = Fasta_Pos["PreSequence"].tolist()
SecondToLastExonSequence = Fasta_Pos["SecondToLastExonSequence"].tolist()
LastIntronSequence = Fasta_Pos["LastIntronSequence"].tolist()
LastExonSequence = Fasta_Pos["LastExonSequence"].tolist()
GeneLength = Fasta_Pos["GeneLength"].tolist()

In [17]:
df = pd.DataFrame(range(-1000,1001),columns=["index"])
for i in range(len(Fasta_Pos)):
    Pre = PreSequence[i] + SecondToLastExonSequence[i]
    LastIntron = LastIntronSequence[i]
    LastExon = LastExonSequence[i]
    
    input_sequence = Pre + LastExon
        
    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    m6AAI_prob = m6AAI_prob.tolist()    
    m6AAI_prob = m6AAI_prob[0:(len(Pre))] + [0]*(len(LastIntron)) + m6AAI_prob[(len(Pre)):]

    
    # Define exon location
    Exon = []
    a = Fasta_Pos.loc[i, "exonStarts"].split(",")[0:-1]
    b = Fasta_Pos.loc[i, "exonEnds"].split(",")[0:-1]
    A = [int(u) for u in a]
    B = [int(v) for v in b]
    Exon = A + B
    Exon = sorted(Exon)
    
    # Define length of exon and intron
    Length = []
    for j in range(1,len(Exon),1):
        length = Exon[j] - Exon[-1 + j]
        Length.append(length)
    
    CumSum = []
    Sum = 0
    for k in Length:
        Sum = Sum + k
        CumSum.append(Sum)

    # Define probability in cDNA
    First_prob = [m6AAI_prob[0:CumSum[0]]]
    for m in range(1,len(CumSum),1):
        prob = m6AAI_prob[(CumSum[m-1]):(CumSum[m])]
        First_prob.append(prob)
    
    Probability = []
    for n in range(0,len(First_prob),2):
        List = First_prob[n]
        Probability.append(List)

    iM6A_prob = []
    for t in Probability:
        iM6A_prob = iM6A_prob + t
        
    Start = 0
    for t in range(len(Probability)-2):
        Start = Start + len(Probability[t])        
    
    Pre = iM6A_prob[0:Start]
    Last = iM6A_prob[Start:]
    
    if len(Pre) < 1000:
        Pre = [0]*(1000-Start) + Pre
    if len(Pre) >= 1000:
        Pre = Pre[-1000:]
    
    if len(Last) < 1001:
        Last = Last + [0]*(1001-len(Last))
    if len(Last) >= 1001:
        Last = Last[0:1001]
    
    New = np.array(Pre + Last)
    New = pd.DataFrame({Fasta_Pos.loc[i,"name"]:New})
    
    df = pd.merge(df, New, left_index=True, right_index=True)        

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


### Select negative strand

In [18]:
Fasta_Neg = Fasta[Fasta["strand"]=="-"]

In [19]:
Fasta_Neg = Fasta_Neg.reset_index(drop = True)

In [20]:
Fasta_Neg

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,...,Group,Sequence,GeneLength,LastExonLength,LastIntronLength,SecondToLastLength,Pre,PreSequence,LastIntronSequence,LastExonSequence
0,Adora1,chr1,-,134199224,134235431,134202950,134234355,3,"134199224,134234014,134235227,","134203590,134234412,134235431,",...,2,ACTCAGCTCCCGCGCCGGGGGCGACGAAGGCGCCGGCGCCAGAGTG...,36207,4366,30424,398,1417,ACTCAGCTCCCGCGCCGGGGGCGACGAAGGCGCCGGCGCCAGAGTG...,GTGAGTTCATAGCAGCCCAAGGTACTCTGTGCAACCCTGTGTGGGG...,GTACAAGACAGTGGTGACTCAGCGGCGGGCAGCAGTGGCCATAGCC...
1,Sntg1,chr1,-,8361474,9299238,8363474,8803943,21,"8361474,8414202,8445026,8447979,8463393,846543...","8363633,8414313,8445119,8448132,8463465,846555...",...,2,CTTCGCGGCGATACAGCCTCTCGGAAACAAAGGTATCCCTAGCCTT...,937764,2159,50569,111,885036,CTTCGCGGCGATACAGCCTCTCGGAAACAAAGGTATCCCTAGCCTT...,GTAAACACAAGAAACCTCCATATAATACATCTCCAATTATGTCAGT...,GAGTTGGAATTTTCAAATTTATTTGCTGTTCTTCACTGTATTCATT...
2,Syt14,chr1,-,192891232,193035775,192897306,193035635,8,"192891232,192901751,192930477,192933113,192980...","192897620,192901941,192930927,192933385,192980...",...,2,GCCCCGCCCCAGCTCCTGGAGGCCCAGCTGCCTGGCCCCGGGGCGT...,144543,6388,4131,190,134024,GCCCCGCCCCAGCTCCTGGAGGCCCAGCTGCCTGGCCCCGGGGCGT...,GTGAGTGAAAAAGAGTTATTTTTATCTTAATGTTTTCGGCACTTAG...,ATACATATGTTAAGTTAACACTGCTGAATTCCATGGGCCAAGAGAT...
3,Lactb2,chr1,-,13623329,13660546,13626860,13660450,7,"13623329,13627355,13630064,13638230,13647384,1...","13626904,13627437,13630213,13638409,13647511,1...",...,2,GTCACGAGACCGCCGCCCTGCCCTCCGAGCCTGAGGTCGCAGCACG...,37217,3575,451,82,33191,GTCACGAGACCGCCGCCCTGCCCTCCGAGCCTGAGGTCGCAGCACG...,GTATGTATCCTTCTGAATATTAATGCCTTTTCTCCTGGCACTTAAA...,TCTACACAACAACTCCTGTCAAGAAATGGAAAGCCGTCCTTTAGTT...
4,Actr1b,chr1,-,36698113,36710008,36700075,36709853,11,"36698113,36700336,36700535,36701133,36701420,3...","36700178,36700377,36700597,36701308,36701513,3...",...,2,GATTGGCCACGGGGCCGCGTTCGCGTAGGCAGCGCACTGGGGGAGG...,11895,2065,158,41,9672,GATTGGCCACGGGGCCGCGTTCGCGTAGGCAGCGCACTGGGGGAGG...,GTGAGATTGGGCTCAAGGAGAAGGGCTCGGGGAGGGACTGGCCGGG...,AGGCTCCATCCTCGCCTCACTGGATACCTTCAAGAAGATGTGGGTA...
5,Lonrf2,chr1,-,38794508,38821215,38798028,38821215,11,"38794508,38800650,38804238,38807040,38807835,3...","38798223,38800800,38804401,38807199,38807957,3...",...,2,ATGAAGAACTATGATCAAGCCCTGCAGGATGCTGAAGCAGTTTGTC...,26707,3715,2427,150,20565,ATGAAGAACTATGATCAAGCCCTGCAGGATGCTGAAGCAGTTTGTC...,GTAGGCTTTCCTTCCTTCTTTCCTGGTGCAGCTCCTCCCCTCAGTC...,AGCAATTCTAGTGGCCCTGCCTGGTCCTGGTGGATTCTGGCCGTGT...
6,Pgap1,chr1,-,54472993,54557682,54480705,54557580,27,"54472993,54481851,54486469,54490220,54492028,5...","54480844,54481956,54486657,54490271,54492164,5...",...,2,AGGCGCGCGCTGCTCAGCTGGGCACGGCTTGGCACCGGAGTCTTTC...,84689,7851,1007,105,75831,AGGCGCGCGCTGCTCAGCTGGGCACGGCTTGGCACCGGAGTCTTTC...,GTACGACAGTTTATTTCTTTCACGTGTCGCTGGATGTTATCACTTT...,TAAATTGCTGAAGACCGTCTCCCAGTTCCCACTTCCTCTGGCTGTT...
7,Erbb4,chr1,-,68032185,69108059,68040040,69107756,27,"68032185,68042440,68071517,68073097,68075417,6...","68040486,68042738,68071688,68073195,68075564,6...",...,2,ACTCCGGGAACTAGCTGTACGTTGTGCTCGGAGCACCAGCCGCACA...,1075874,8301,1954,298,1065619,ACTCCGGGAACTAGCTGTACGTTGTGCTCGGAGCACCAGCCGCACA...,GTACAAGCTGAATGTTCTGAAGTGATTGTGTTCAATGATCACTGCC...,AATATCTGAATCCTGTGGAAGAGAACCCTTTTGTGTCCCGAAGGAA...
8,Dpp10,chr1,-,123321470,124045559,123334248,124045443,26,"123321470,123336823,123341104,123341319,123341...","123334383,123336897,123341177,123341378,123341...",...,2,TCCGCCAATTCCGAGAGCGACGGGGCGTCTGTGACATGCGATCGCT...,724089,12913,2440,74,708736,TCCGCCAATTCCGAGAGCGACGGGGCGTCTGTGACATGCGATCGCT...,GTAAGCCACCACTCCAGAAAAATGAGTACTAATTCttagctttata...,GTCTATCCAGATGAAGGATATCACATTTCAGACAAGAGCAAGCATC...
9,Cdk18,chr1,-,132112236,132139684,132114935,132122435,16,"132112236,132115464,132115861,132116446,132116...","132114970,132115542,132115952,132116489,132116...",...,2,GCCTTCCTCCTCCTCTTACCCACCTTCCTGCACCCCTCCCTTCTCC...,27448,2734,494,78,24220,GCCTTCCTCCTCCTCTTACCCACCTTCCTGCACCCCTCCCTTCTCC...,GTAGGGCCA

In [21]:
PreSequence = Fasta_Neg["PreSequence"].tolist()
SecondToLastExonSequence = Fasta_Neg["SecondToLastExonSequence"].tolist()
LastIntronSequence = Fasta_Neg["LastIntronSequence"].tolist()
LastExonSequence = Fasta_Neg["LastExonSequence"].tolist()
GeneLength = Fasta_Neg["GeneLength"].tolist()

In [22]:
for i in range(len(Fasta_Neg)):
    Pre = PreSequence[i] + SecondToLastExonSequence[i]
    LastIntron = LastIntronSequence[i]
    LastExon = LastExonSequence[i]
    
    input_sequence = Pre + LastExon
        
    x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
    y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
    m6AAI_prob = y[0, :, 1]
    m6AAI_prob = m6AAI_prob.tolist()    
    m6AAI_prob = m6AAI_prob[0:(len(Pre))] + [0]*(len(LastIntron)) + m6AAI_prob[(len(Pre)):]

    
    # Define exon location
    Exon = []
    a = Fasta_Neg.loc[i, "exonStarts"].split(",")[0:-1]
    b = Fasta_Neg.loc[i, "exonEnds"].split(",")[0:-1]
    A = [int(u) for u in a]
    B = [int(v) for v in b]
    Exon = A + B
    Exon = sorted(Exon)
    
    # Define length of exon and intron
    Length = []
    for j in range(1,len(Exon),1):
        length = Exon[j] - Exon[-1 + j]
        Length.append(length)
    Length = Length[::-1]
    
    CumSum = []
    Sum = 0
    for k in Length:
        Sum = Sum + k
        CumSum.append(Sum)

    # Define probability in cDNA
    First_prob = [m6AAI_prob[0:CumSum[0]]]
    for m in range(1,len(CumSum),1):
        prob = m6AAI_prob[(CumSum[m-1]):(CumSum[m])]
        First_prob.append(prob)
    
    Probability = []
    for n in range(0,len(First_prob),2):
        List = First_prob[n]
        Probability.append(List)

    iM6A_prob = []
    for t in Probability:
        iM6A_prob = iM6A_prob + t
        
    Start = 0
    for t in range(len(Probability)-2):
        Start = Start + len(Probability[t])        
    
    Pre = iM6A_prob[0:Start]
    Last = iM6A_prob[Start:]
    
    if len(Pre) < 1000:
        Pre = [0]*(1000-Start) + Pre
    if len(Pre) >= 1000:
        Pre = Pre[-1000:]
    
    if len(Last) < 1001:
        Last = Last + [0]*(1001-len(Last))
    if len(Last) >= 1001:
        Last = Last[0:1001]
    
    New = np.array(Pre + Last)
    New = pd.DataFrame({Fasta_Neg.loc[i,"name"]:New})
    
    df = pd.merge(df, New, left_index=True, right_index=True)        

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


In [23]:
df

,index,Cflar,Lmx1a,Gpr45,Plcl1,Bmpr2,Pard3b,Cntnap5a,Clasp1,Nmnat2,...,Ppp1r3c,Ide,Tm9sf3,Arhgap19,Cox15,Cwf19l1,Scd1,Mgea5,Pdzd8,Rab11fip2
0,-1000,0.000000e+00,2.488617e-11,0.000000e+00,4.595343e-08,1.086154e-12,4.810022e-12,1.811271e-08,1.271960e-12,0.000000e+00,...,0.000000e+00,1.706238e-11,1.888094e-12,1.022746e-11,6.161762e-10,1.971134e-11,0.000000e+00,8.478808e-08,1.135312e-09,4.259565e-12
1,-999,0.000000e+00,2.035221e-10,0.000000e+00,2.318347e-09,1.809121e-12,2.047634e-11,5.501527e-10,8.909876e-13,0.000000e+00,...,0.000000e+00,2.923681e-11,2.313691e-12,1.756299e-11,9.006405e-10,8.733521e-11,0.000000e+00,3.843836e-05,1.767584e-10,4.250643e-12
2,-998,0.000000e+00,1.117630e-10,0.000000e+00,3.254204e-09,2.240561e-12,1.131916e-11,5.073384e-09,1.315294e-11,0.000000e+00,...,0.000000e+00,9.895568e-12,2.145017e-10,1.802264e-09,1.472260e-10,2.016939e-11,0.000000e+00,4.590037e-07,9.934383e-10,7.795370e-12
3,-997,0.000000e+00,2.799468e-11,0.000000e+00,5.232358e-09,3.940770e-12,8.211822e-12,2.142975e-08,9.157931e-12,0.000000e+00,...,0.000000e+00,1.352939e-09,9.749447e-11,1.811781e-11,2.349754e-10,1.905752e-11,0.000000e+00,2.594786e-05,7.578617e-10,4.352481e-12
4,-996,0.000000e+00,1.303933e-10,0.000000e+00,2.005925e-08,1.321958e-12,7.711016e-12,4.607824e-09,4.137270e-12,0.000000e+00,...,0.000000e+00,2.880154e-11,4.498609e-12,2.389313e-10,6.802298e-10,1.367607e-09,0.000000e+00,2.075708e-06,3.060169e-10,4.872287e-12
5,-995,0.000000e+00,3.563007e-10,0.000000e+00,6.667499e-07,7.836806e-12,1.029742e-10,1.020560e-04,1.973939e-12,0.000000e+00,...,0.000000e+00,7.542920e-11,7.690020e-12,3.801447e-11,2.876335e-10,1.845170e-11,0.000000e+00,1.291668e-06,1.150555e-09,2.432581e-12
6,-994,0.000000e+00,2.318778e-10,0.000000e+00,2.594832e-09,4.703254e-12,1.490069e-10,4.938978e-08,3.125969e-12,0.000000e+00,...,0.000000e+00,1.581496e-09,3.579910e-11,1.054965e-10,4.275023e-10,1.185019e-11,0.000000e+00,2.664772e-07,6.445632e-09,1.004229e-11
7,-993,0.000000e+00,3.968968e-11,0.000000e+00,1.511350e-09,1.216962e-11,6.934509e-12,1.338773e-09,1.458690e-11,0.000000e+00,...,0.000000e+00,1.744248e-10,5.721757e-12,1.053123e-10,1.720147e-09,4.754024e-11,0.000000e+00,5.908618e-07,3.713197e-09,4.783624e-12
8,-992,0.000000e+00,1.519511e-09,0.000000e+00,2.327038e-07,7.832581e-07,2.086212e-11,1.320377e-09,2.001254e-12,0.000000e+00,...,0.000000e+00,1.689571e-10,8.404324e-12,1.331730e-11,1.214329e-04,2.026380e-11,0.000000e+00,1.994880e-07,1.925728e-09,5.503592e-12
9,-991,0.000000e+00,1.656024e-10,0.000000e+00,5.913980e-09,1.658297e-09,6.880497e-12,5.910908e-10,6.019462e-12,0.000000e+00,...,0.000000e+00,3.818076e-06,3.868428e-12,8.873442e-12,2.109277e-09,1.265312e-11,0.000000e+00,2.258930e-07,6.396502e-09,2.405730e-11


In [ ]:
df.to_csv("iM6A_mouse_SecondToLastExonStart_LastIntronDeletion_2genes.csv", index=0)

In [24]:
df.drop(["index"], axis=1, inplace=True)
df = df.T

In [25]:
df.loc["Sum"] = df.sum()
df = df.T

In [26]:
Value = df[["Sum"]]

In [27]:
Value.head(5)

,Sum
0,1.274780
1,0.671743
2,2.088128
3,1.247576
4,2.088729


In [28]:
df.drop(["Sum"], axis=1, inplace=True)
df = df.T

In [29]:
df.loc["Number"] = (df > 0).sum()
df = df.T

In [30]:
Number = df[["Number"]]

In [31]:
Number.head(5)

,Number
0,2688.0
1,2689.0
2,2692.0
3,2693.0
4,2694.0


In [32]:
Data = pd.concat([Value, Number], axis=1)

In [33]:
Data["Mean"] = Data["Sum"]/Data["Number"]

In [34]:
Data

,Sum,Number,Mean
0,1.274780,2688.0,0.000474
1,0.671743,2689.0,0.000250
2,2.088128,2692.0,0.000776
3,1.247576,2693.0,0.000463
4,2.088729,2694.0,0.000775
5,1.035966,2696.0,0.000384
6,1.976643,2697.0,0.000733
7,2.117111,2697.0,0.000785
8,1.397858,2702.0,0.000517
9,1.182765,2703.0,0.000438


In [36]:
Data = Data.head(2000)
Sum = []
Number = []

for j in range(0,len(Data),10):

    a = Data.loc[j:(j+9),"Sum"]
    b = np.mean(a, axis=0)
    Sum.append(b)
    
    a = Data.loc[j:(j+9),"Number"]
    b = np.mean(a, axis=0)
    Number.append(b)

Result = pd.DataFrame({"Index":range(-1000,1000,10), "Sum":Sum, "Number":Number})
Result["Mean"] = Result["Sum"]/Result["Number"] 

In [37]:
Result

,Index,Number,Sum,Mean
0,-1000,2695.1,1.508130,0.000560
1,-990,2713.9,2.195369,0.000809
2,-980,2729.1,2.437126,0.000893
3,-970,2746.8,1.612347,0.000587
4,-960,2763.6,2.071008,0.000749
5,-950,2775.1,1.807106,0.000651
6,-940,2791.9,1.970138,0.000706
7,-930,2811.8,1.695074,0.000603
8,-920,2833.0,1.575444,0.000556
9,-910,2856.0,1.523036,0.000533


In [38]:
Result.to_csv("iM6A_mouse_SecondToLastExonStart_LastIntronDeletion.csv", index=0)